## Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import optuna
from optuna.integration import TFKerasPruningCallback
from optuna.trial import TrialState
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_contour
from sklearn.preprocessing import MinMaxScaler
from torchmetrics.functional.classification import *
import tensorflow as tf
from tensorflow import keras
from keras.utils import *
from keras.layers import *
from keras.models import *
from keras.callbacks import *
from skmultilearn.ext import Keras

## Read Data

In [2]:
data = pd.read_csv('D:\Project\project-ednn\Code\Final_dataset_Diabetes_Complication.csv')
data.head()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X24,X25,X26,Y1,Y2,Y3,Y4,Y5,Y6,Y7
0,3,1,1,1,1,1,1,1,3,1,...,1,3,1,2,2,1,2,2,1,2
1,5,3,1,1,2,4,2,1,5,1,...,1,2,2,2,2,2,2,2,1,1
2,4,1,1,1,2,1,2,2,2,2,...,2,3,1,2,2,2,2,2,2,1
3,4,1,1,1,2,4,2,1,1,1,...,2,3,2,2,2,2,1,2,1,2
4,2,1,1,3,1,3,1,1,1,1,...,1,3,1,2,2,2,2,2,2,2


In [3]:
data = data.drop_duplicates(ignore_index=True)
data

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X24,X25,X26,Y1,Y2,Y3,Y4,Y5,Y6,Y7
0,3,1,1,1,1,1,1,1,3,1,...,1,3,1,2,2,1,2,2,1,2
1,5,3,1,1,2,4,2,1,5,1,...,1,2,2,2,2,2,2,2,1,1
2,4,1,1,1,2,1,2,2,2,2,...,2,3,1,2,2,2,2,2,2,1
3,4,1,1,1,2,4,2,1,1,1,...,2,3,2,2,2,2,1,2,1,2
4,2,1,1,3,1,3,1,1,1,1,...,1,3,1,2,2,2,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
543770,1,1,2,1,1,4,1,1,5,2,...,2,3,1,2,2,2,2,2,2,2
543771,2,1,1,1,1,4,1,1,5,1,...,1,2,2,2,2,1,2,2,2,1
543772,1,2,1,1,1,3,1,1,2,1,...,1,3,1,2,2,2,2,2,2,1
543773,3,1,2,1,1,3,1,1,2,2,...,1,3,2,2,2,2,2,1,2,2


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 543775 entries, 0 to 543774
Data columns (total 33 columns):
 #   Column  Non-Null Count   Dtype
---  ------  --------------   -----
 0   X1      543775 non-null  int64
 1   X2      543775 non-null  int64
 2   X3      543775 non-null  int64
 3   X4      543775 non-null  int64
 4   X5      543775 non-null  int64
 5   X6      543775 non-null  int64
 6   X7      543775 non-null  int64
 7   X8      543775 non-null  int64
 8   X9      543775 non-null  int64
 9   X10     543775 non-null  int64
 10  X11     543775 non-null  int64
 11  X12     543775 non-null  int64
 12  X13     543775 non-null  int64
 13  X14     543775 non-null  int64
 14  X15     543775 non-null  int64
 15  X16     543775 non-null  int64
 16  X17     543775 non-null  int64
 17  X18     543775 non-null  int64
 18  X19     543775 non-null  int64
 19  X20     543775 non-null  int64
 20  X21     543775 non-null  int64
 21  X22     543775 non-null  int64
 22  X23     543775 non-n

## Data Visualization

In [5]:
data_feature = data.iloc[:, :26]
data_label = data.iloc[:, 26:]
data_label

,Y1,Y2,Y3,Y4,Y5,Y6,Y7
0,2,2,1,2,2,1,2
1,2,2,2,2,2,1,1
2,2,2,2,2,2,2,1
3,2,2,2,1,2,1,2
4,2,2,2,2,2,2,2
...,...,...,...,...,...,...,...
543770,2,2,2,2,2,2,2
543771,2,2,1,2,2,2,1
543772,2,2,2,2,2,2,1
543773,2,2,2,2,1,2,2


## Data Preprocessing

In [6]:
scaler = MinMaxScaler()
scaler.fit(data)

# Transform the DataFrame to obtain the normalized data
data_normalized = scaler.transform(data)

In [7]:
X = data_normalized[:, :26]
y = data_normalized[:, 26:]

In [8]:
data_normalize = pd.DataFrame(scaler.transform(data), columns=data.columns)
df_feature = data_normalize.iloc[:, :26]
df_label = data_normalize.iloc[:, 26:]

## Modelling

In [9]:
weight_list = []
for i, label in enumerate(df_label.columns):
    values_1 = df_label[label][df_label[label] == 1]
    values_2 = df_label[label][df_label[label] == 2]
    dist = 1 - (len(values_1) / len(df_label))
    weight_list.append(dist)

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [11]:
weights_ = torch.Tensor(weight_list).to(device)
weights_

tensor([0.0483, 0.0652, 0.0605, 0.0508, 0.1008, 0.3895, 0.4290],
       device='cuda:0')

In [12]:
def reset_weights(m):
    for layer in m.children():
        if hasattr(layer, 'reset_parameters'):
            # print(f'Reset trainable parameters of layer = {layer}')
            layer.reset_parameters()

In [13]:
class DNNModelwithWeightedDropout(tf.keras.Model):
    def __init__(self, num_features=26, num_classes=7, p=[0.05, 0.01, 0.003]):
        super(DNNModelwithWeightedDropout, self).__init__()
        self.fcn1 = tf.keras.Sequential([
            tf.keras.layers.Dense(32, activation='relu', input_shape=(num_features,)),
            tf.keras.layers.Dense(len(p), activation='softmax')
        ])
        self.dropout = [tf.keras.layers.Dropout(rate=rate) for rate in p]
        self.fcn2 = tf.keras.Sequential([
            tf.keras.layers.Dense(num_classes, activation='sigmoid')
        ])

    def call(self, x, training=None, mask=None):
        x = self.fcn1(x)
        weights = x

        # Apply weighted dropout
        x = [dropout(x[:, i], training=training) * weights[:, i] for i, dropout in enumerate(self.dropout)]

        x = tf.stack(x, axis=1)
        x = self.fcn2(x)
        return x

In [14]:
class DNNDropout(tf.keras.Model):
    def __init__(self, num_features=26, num_classes=7, p=0.5):
        super(DNNDropout, self).__init__()

        self.fcn1 = tf.keras.Sequential([
            tf.keras.layers.Dense(16, activation='relu'),
            tf.keras.layers.Dense(8, activation='relu'),
            tf.keras.layers.Dense(3, activation='softmax')
        ])

        self.dropout1 = tf.keras.layers.Dropout(p)

        self.fcn2 = tf.keras.Sequential([
            tf.keras.layers.Dense(num_classes, activation='sigmoid')
        ])

    def call(self, x, training=False):
        x = self.fcn1(x)
        x = self.dropout1(x)
        x = self.fcn2(x)
        return x

In [15]:
def DropoutModel(trial):
    dropout_rate = trial.suggest_float('dropout_rate', 0.0, 0.5)
    model = DNNDropout(num_features=26, num_classes=7, p=dropout_rate)
    lossWeights = weight_list
    model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.003), metrics=['accuracy'], loss_weights=lossWeights)

    return model

In [16]:
def ProposedModel(trial):
    dropout_rate1 = trial.suggest_float('dropout_rate1', 0.0, 0.5)
    dropout_rate2 = trial.suggest_float('dropout_rate2', 0.0, 0.5)
    dropout_rate3 = trial.suggest_float('dropout_rate3', 0.0, 0.5)
    dropout_rate = [dropout_rate1, dropout_rate2, dropout_rate3]
    model = DNNModelwithWeightedDropout(num_features=26, num_classes=7, p=dropout_rate)
    lossWeights = weight_list
    model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.003), metrics=['accuracy'], loss_weights=lossWeights)

    return model

## Hyperparameter Tuning

In [17]:
from sklearn.model_selection import train_test_split
x_train, x_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
x_test, x_valid, y_test, y_valid = train_test_split(x_temp, y_temp, test_size=0.3, random_state=42)

In [18]:
def objective1(trial):
    keras.backend.clear_session()
    model = DropoutModel(trial)

    model.fit(
        x_train,
        y_train,
        batch_size=32,
        epochs=10,
        callbacks=[TFKerasPruningCallback(trial, "val_loss")],
        validation_data=(x_valid, y_valid),
        verbose=1,
    )

    # Evaluate the model accuracy on the validation set.
    prediction = model.predict(x_test)
    trues = torch.tensor(y_test)
    preds = torch.tensor(prediction)

    score = multilabel_accuracy(preds, trues, num_labels=7, average='micro').item()
    return score

In [19]:
def objective2(trial):
    keras.backend.clear_session()
        # Generate our trial model.
    model = ProposedModel(trial)

    # Fit the model on the training data.
    # The TFKerasPruningCallback checks for pruning condition every epoch.
    model.fit(
        x_train,
        y_train,
        batch_size=32,
        epochs=10,
        callbacks=[TFKerasPruningCallback(trial, "val_loss")],
        validation_data=(x_valid, y_valid),
        verbose=1,
    )

    # Evaluate the model accuracy on the validation set.
    prediction = model.predict(x_test)
    trues = torch.tensor(y_test)
    preds = torch.tensor(prediction)

    score = multilabel_accuracy(preds, trues, num_labels=7, average='micro').item()
    return score

In [20]:
study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.HyperbandPruner())
study.optimize(objective1, n_trials=50)
pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

[I 2024-01-24 23:51:36,076] A new study created in memory with name: no-name-bcf301fb-265f-431f-a1ae-6938ea11488c


Epoch 1/10
11896/11896 [==============================] - 45s 4ms/step - loss: 0.0159 - accuracy: 0.3464 - val_loss: 0.0149 - val_accuracy: 0.3023
Epoch 2/10
11896/11896 [==============================] - 42s 3ms/step - loss: 0.0155 - accuracy: 0.3178 - val_loss: 0.0150 - val_accuracy: 0.1866
Epoch 3/10
11896/11896 [==============================] - 43s 4ms/step - loss: 0.0155 - accuracy: 0.3142 - val_loss: 0.0150 - val_accuracy: 0.1669
Epoch 4/10
11896/11896 [==============================] - 41s 3ms/step - loss: 0.0155 - accuracy: 0.3304 - val_loss: 0.0149 - val_accuracy: 0.3758
Epoch 5/10
11896/11896 [==============================] - 43s 4ms/step - loss: 0.0155 - accuracy: 0.3341 - val_loss: 0.0150 - val_accuracy: 0.2193
Epoch 6/10
11896/11896 [==============================] - 42s 4ms/step - loss: 0.0155 - accuracy: 0.3060 - val_loss: 0.0149 - val_accuracy: 0.3001
Epoch 7/10
11896/11896 [==============================] - 44s 4ms/step - loss: 0.0155 - accuracy: 0.3204 - val_loss: 0

[I 2024-01-24 23:58:46,668] Trial 0 finished with value: 0.8695766925811768 and parameters: {'dropout_rate': 0.1576467440183411}. Best is trial 0 with value: 0.8695766925811768.


Epoch 1/10
11896/11896 [==============================] - 41s 3ms/step - loss: 0.0163 - accuracy: 0.3668 - val_loss: 0.0154 - val_accuracy: 0.0059
Epoch 2/10
11896/11896 [==============================] - 41s 3ms/step - loss: 0.0160 - accuracy: 0.3973 - val_loss: 0.0154 - val_accuracy: 0.3945
Epoch 3/10
11896/11896 [==============================] - 41s 3ms/step - loss: 0.0160 - accuracy: 0.4009 - val_loss: 0.0155 - val_accuracy: 0.2107
Epoch 4/10
11896/11896 [==============================] - 41s 3ms/step - loss: 0.0160 - accuracy: 0.3971 - val_loss: 0.0155 - val_accuracy: 0.3315
Epoch 5/10
11896/11896 [==============================] - 42s 3ms/step - loss: 0.0160 - accuracy: 0.3952 - val_loss: 0.0154 - val_accuracy: 0.2534
Epoch 6/10
11896/11896 [==============================] - 41s 3ms/step - loss: 0.0160 - accuracy: 0.3850 - val_loss: 0.0154 - val_accuracy: 0.3929
Epoch 7/10
11896/11896 [==============================] - 41s 3ms/step - loss: 0.0160 - accuracy: 0.4111 - val_loss: 0

[I 2024-01-25 00:05:40,061] Trial 1 finished with value: 0.8618667125701904 and parameters: {'dropout_rate': 0.343185432370909}. Best is trial 0 with value: 0.8695766925811768.


...

[I 2024-01-25 02:56:50,961] Trial 47 pruned. Trial was pruned at epoch 9.


Epoch 1/10
11896/11896 [==============================] - 42s 3ms/step - loss: 0.0165 - accuracy: 0.4273 - val_loss: 0.0155 - val_accuracy: 0.3673
Epoch 2/10
11896/11896 [==============================] - 41s 3ms/step - loss: 0.0160 - accuracy: 0.3970 - val_loss: 0.0155 - val_accuracy: 0.4491
Epoch 3/10
11896/11896 [==============================] - 41s 3ms/step - loss: 0.0160 - accuracy: 0.3946 - val_loss: 0.0155 - val_accuracy: 0.3203
Epoch 4/10
11881/11896 [============================>.] - ETA: 0s - loss: 0.0160 - accuracy: 0.3841

[I 2024-01-25 02:59:36,866] Trial 48 pruned. Trial was pruned at epoch 3.


Epoch 1/10
11896/11896 [==============================] - 42s 4ms/step - loss: 0.0160 - accuracy: 0.3446 - val_loss: 0.0153 - val_accuracy: 0.3384
Epoch 2/10
11896/11896 [==============================] - 42s 3ms/step - loss: 0.0157 - accuracy: 0.3639 - val_loss: 0.0152 - val_accuracy: 0.2185
Epoch 3/10
11896/11896 [==============================] - 42s 3ms/step - loss: 0.0157 - accuracy: 0.3552 - val_loss: 0.0151 - val_accuracy: 0.3928
Epoch 4/10
11885/11896 [============================>.] - ETA: 0s - loss: 0.0157 - accuracy: 0.3710

[I 2024-01-25 03:02:23,827] Trial 49 pruned. Trial was pruned at epoch 3.


In [21]:
print("\nStudy statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

trial = study.best_trial
print("Best trial:")
print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

# Save results to csv file
df = study.trials_dataframe().drop(['datetime_start', 'datetime_complete', 'duration'], axis=1)  # Exclude columns
df = df.loc[df['state'] == 'COMPLETE']        # Keep only results that did not prune
df = df.drop('state', axis=1)                 # Exclude state column
df = df.sort_values('value')                  # Sort based on accuracy
df.to_csv('optuna_results.csv', index=False)  # Save to csv file

# Display results in a dataframe
print("\nOverall Results (ordered by accuracy):\n {}".format(df))

# Find the most important hyperparameters
most_important_parameters = optuna.importance.get_param_importances(study, target=None)

# Display the most important hyperparameters
print('\nMost important hyperparameters:')
for key, value in most_important_parameters.items():
    print('  {}:{}{:.2f}%'.format(key, (15-len(key))*' ', value*100))


Study statistics: 
  Number of finished trials:  50
  Number of pruned trials:  42
  Number of complete trials:  8
Best trial:
  Value:  0.8712630867958069
  Params: 
    dropout_rate: 0.07899167787544625

Overall Results (ordered by accuracy):
     number     value  params_dropout_rate  system_attrs_completed_rung_0  \
2        2  0.854060             0.459002                       0.015597   
7        7  0.861270             0.286755                       0.015333   
1        1  0.861867             0.343185                       0.015438   
30      30  0.862021             0.300665                       0.015360   
28      28  0.867245             0.180687                       0.015275   
0        0  0.869577             0.157647                            NaN   
8        8  0.869789             0.180184                       0.015075   
6        6  0.871263             0.078992                       0.014836   

    system_attrs_completed_rung_1  system_attrs_completed_rung_2  
2

In [22]:
study2 = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.HyperbandPruner())
study2.optimize(objective2, n_trials=50)
pruned_trials2 = study2.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials2 = study2.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

[I 2024-01-25 03:02:23,961] A new study created in memory with name: no-name-d343fe65-fc72-4d4b-99a9-2a0a962c4932


Epoch 1/10
11896/11896 [==============================] - 44s 4ms/step - loss: 0.0160 - accuracy: 0.3324 - val_loss: 0.0151 - val_accuracy: 0.4301
Epoch 2/10
11896/11896 [==============================] - 43s 4ms/step - loss: 0.0154 - accuracy: 0.3300 - val_loss: 0.0150 - val_accuracy: 0.3216
Epoch 3/10
11896/11896 [==============================] - 43s 4ms/step - loss: 0.0154 - accuracy: 0.3473 - val_loss: 0.0151 - val_accuracy: 0.2634
Epoch 4/10
11896/11896 [==============================] - 43s 4ms/step - loss: 0.0154 - accuracy: 0.3448 - val_loss: 0.0150 - val_accuracy: 0.4310
Epoch 5/10
11896/11896 [==============================] - 43s 4ms/step - loss: 0.0154 - accuracy: 0.3409 - val_loss: 0.0150 - val_accuracy: 0.1537
Epoch 6/10
11896/11896 [==============================] - 44s 4ms/step - loss: 0.0154 - accuracy: 0.3506 - val_loss: 0.0150 - val_accuracy: 0.2725
Epoch 7/10
11896/11896 [==============================] - 46s 4ms/step - loss: 0.0154 - accuracy: 0.3400 - val_loss: 0

[I 2024-01-25 03:09:47,848] Trial 0 finished with value: 0.8676701188087463 and parameters: {'dropout_rate1': 0.49876331222588766, 'dropout_rate2': 0.12300947214747354, 'dropout_rate3': 0.07889421808424685}. Best is trial 0 with value: 0.8676701188087463.


Epoch 1/10
11896/11896 [==============================] - 45s 4ms/step - loss: 0.0160 - accuracy: 0.3698 - val_loss: 0.0151 - val_accuracy: 0.2612
Epoch 2/10
11896/11896 [==============================] - 43s 4ms/step - loss: 0.0155 - accuracy: 0.3548 - val_loss: 0.0151 - val_accuracy: 0.3331
Epoch 3/10
11896/11896 [==============================] - 43s 4ms/step - loss: 0.0155 - accuracy: 0.3592 - val_loss: 0.0151 - val_accuracy: 0.1279
Epoch 4/10
11896/11896 [==============================] - 44s 4ms/step - loss: 0.0155 - accuracy: 0.3561 - val_loss: 0.0151 - val_accuracy: 0.3040
Epoch 5/10
11896/11896 [==============================] - 44s 4ms/step - loss: 0.0155 - accuracy: 0.3566 - val_loss: 0.0151 - val_accuracy: 0.2573
Epoch 6/10
11896/11896 [==============================] - 44s 4ms/step - loss: 0.0155 - accuracy: 0.3560 - val_loss: 0.0150 - val_accuracy: 0.4362
Epoch 7/10
11896/11896 [==============================] - 44s 4ms/step - loss: 0.0155 - accuracy: 0.3533 - val_loss: 0

[I 2024-01-25 03:17:10,163] Trial 1 finished with value: 0.8692013621330261 and parameters: {'dropout_rate1': 0.10881628315448705, 'dropout_rate2': 0.21645253628714312, 'dropout_rate3': 0.28275113817466674}. Best is trial 1 with value: 0.8692013621330261.


...

[I 2024-01-25 06:23:43,847] Trial 47 pruned. Trial was pruned at epoch 1.


Epoch 1/10
11896/11896 [==============================] - 44s 4ms/step - loss: 0.0160 - accuracy: 0.3778 - val_loss: 0.0151 - val_accuracy: 0.4258
Epoch 2/10
11896/11896 [==============================] - ETA: 0s - loss: 0.0156 - accuracy: 0.3858

[I 2024-01-25 06:25:10,615] Trial 48 pruned. Trial was pruned at epoch 1.


Epoch 1/10
11896/11896 [==============================] - 43s 4ms/step - loss: 0.0159 - accuracy: 0.4188 - val_loss: 0.0151 - val_accuracy: 0.2416
Epoch 2/10
11896/11896 [==============================] - 43s 4ms/step - loss: 0.0155 - accuracy: 0.3908 - val_loss: 0.0151 - val_accuracy: 0.1965
Epoch 3/10
11896/11896 [==============================] - 43s 4ms/step - loss: 0.0155 - accuracy: 0.3883 - val_loss: 0.0151 - val_accuracy: 0.3161
Epoch 4/10
11896/11896 [==============================] - 43s 4ms/step - loss: 0.0155 - accuracy: 0.4104 - val_loss: 0.0150 - val_accuracy: 0.3102
Epoch 5/10
11896/11896 [==============================] - 43s 4ms/step - loss: 0.0155 - accuracy: 0.3940 - val_loss: 0.0150 - val_accuracy: 0.2629
Epoch 6/10
11896/11896 [==============================] - 43s 4ms/step - loss: 0.0155 - accuracy: 0.3894 - val_loss: 0.0150 - val_accuracy: 0.3353
Epoch 7/10
11896/11896 [==============================] - 43s 4ms/step - loss: 0.0155 - accuracy: 0.3946 - val_loss: 0

[I 2024-01-25 06:32:24,022] Trial 49 pruned. Trial was pruned at epoch 9.


In [23]:
print("\nStudy statistics: ")
print("  Number of finished trials: ", len(study2.trials))
print("  Number of pruned trials: ", len(pruned_trials2))
print("  Number of complete trials: ", len(complete_trials2))

trial = study2.best_trial
print("Best trial:")
print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

# Save results to csv file
df = study2.trials_dataframe().drop(['datetime_start', 'datetime_complete', 'duration'], axis=1)  # Exclude columns
df = df.loc[df['state'] == 'COMPLETE']        # Keep only results that did not prune
df = df.drop('state', axis=1)                 # Exclude state column
df = df.sort_values('value')                  # Sort based on accuracy
df.to_csv('optuna_results.csv', index=False)  # Save to csv file

# Display results in a dataframe
print("\nOverall Results (ordered by accuracy):\n {}".format(df))

# Find the most important hyperparameters
most_important_parameters = optuna.importance.get_param_importances(study2, target=None)

# Display the most important hyperparameters
print('\nMost important hyperparameters:')
for key, value in most_important_parameters.items():
    print('  {}:{}{:.2f}%'.format(key, (15-len(key))*' ', value*100))


Study statistics: 
  Number of finished trials:  50
  Number of pruned trials:  40
  Number of complete trials:  10
Best trial:
  Value:  0.8707488775253296
  Params: 
    dropout_rate1: 0.2822561966848435
    dropout_rate2: 0.1877127859630215
    dropout_rate3: 0.05954025204453273

Overall Results (ordered by accuracy):
     number     value  params_dropout_rate1  params_dropout_rate2  \
13      13  0.861536              0.383348              0.392737   
2        2  0.862617              0.337636              0.317059   
44      44  0.862750              0.409123              0.419755   
17      17  0.863465              0.426108              0.228223   
45      45  0.863643              0.364301              0.298729   
11      11  0.864769              0.357665              0.327757   
0        0  0.867670              0.498763              0.123009   
1        1  0.869201              0.108816              0.216453   
6        6  0.870061              0.365696              0.35024